# Project nº1 - pySpark

Use pySpark to create indexes for answering the following queries:


## 1.What is the accumulated number of taxi trips per month?

Output is expected to have two columns: (month_number, #total_trips). In the example below
the values are not real and you may get different ones. 

In [1]:
%%time
import pyspark
from operator import add as sum

sc = pyspark.SparkContext('local[*]')
try :
    lines = sc.textFile('Taxi_Trips_151MB.csv')
    non_empty_lines = lines.filter( lambda line : len(line) > 0 )
    date = non_empty_lines.map( lambda line : line.split(';')[2])
    month = date.map( lambda line : line.split('/')[0])
    month_tuples = month.map ( lambda word: (word, 1))
    occurrences = month_tuples.reduceByKey(sum)
    result = occurrences.sortBy( lambda x: x[1], ascending = False)
    for (k,v) in result.take(12):
        print(k,v)
    
    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

03 35260
06 35016
05 34979
10 33618
04 32884
08 32747
07 32141
09 31466
02 31013
01 30357
11 30017
12 29252
CPU times: user 680 ms, sys: 782 ms, total: 1.46 s
Wall time: 10.5 s


##  2.For each pickup region, report the list of unique dropoff regions?

Output is expected to have two columns: (pickup_region_ID, list_of_dropoff_region_ID). In the
example below the values are not real and you should get different ones (the output is nicely
wrapped in this text to facilitate reading).

In [2]:
%%time
import pyspark
from operator import add

sc = pyspark.SparkContext('local[*]')
try :
    lines = sc.textFile('Taxi_Trips_151MB.csv')
    #existir log
    no_empty_logs =lines.filter(lambda lines : len(lines) > 0 )
    #get the date and time
    columns=no_empty_logs.map(lambda log: log.split(';')).map(lambda v:(v[6],v[7]))
    filtering=columns.filter(lambda v: len(v[0])>0 and len(v[1])>0)
    all_clean=filtering.map(lambda v:(v[0],{v[1]}))
    result= all_clean.reduceByKey( lambda a, b: a | b)
    for (k,v) in result.collect():
        print("%s\t%s\n"%(k,v))
        sc.stop()
except Exception as e:
    print(e)
    sc.stop()


17031839100	{'17031310300', '17031243400', '17031243200', '17031062300', '17031040700', '17031241400', '17031842300', '17031320100', '17031030601', '17031283100', '17031071000', '17031081000', '17031241200', '17031770700', '17031061100', '17031081402', '17031060900', '17031071400', '17031080202', '17031160800', '17031140301', '17031081100', '17031030800', '17031841100', '17031831100', '17031010400', '17031031000', '17031410900', '17031030703', '17031841900', '17031062200', '17031062900', '17031811000', '17031060200', '17031241300', '17031221400', '17031351000', '17031837800', '17031010502', '17031060300', '17031070300', '17031063000', '17031839700', '17031221200', '17031081201', '17031010600', '17031051000', '17031843700', '17031242000', '17031050200', '17031032100', '17031843500', '17031020901', '17031283200', '17031031900', '17031031502', '17031060500', '17031839000', '17031411000', '17031080400', '17031840200', '17031832200', '17031842900', '17031830600', '17031242200', '17031063301

## 3.What is the expected charge/cost of a taxi ride, given the pickup region ID, the weekday(0=Monday, 6=Sunday) and time in format “hour AM/PM”?

Output is expected to have two columns: (month_number, avg_total_trip_cost). In the example
below the values are not real and you should get different ones. 

In [3]:
%%time
import pyspark
from operator import add
import datetime
import statistics as statis

#id pick up region [6]
#date [3]
#cost [14]
sc = pyspark.SparkContext('local[*]')
try :
    lines = sc.textFile('Taxi_Trips_151MB.csv')
    #existir log
    no_empty_logs =lines.filter(lambda lines : len(lines) > 0 )
    take_off_comas=no_empty_logs.map(lambda log: log.split(';'))
    filtering=take_off_comas.filter(lambda v: len(v[6])>0 and len(v[2])>0 and len(v[14])>0 )
    
    #get the date and time
    columns=filtering.map(lambda v:(v[6]+"_"+str(datetime.date(int(v[2][6:-11]),int(v[2][:-20]),int(v[2][3:-17])).weekday())+"_"+v[2][11:-9]+v[2][20:],{float(v[14].replace(',',''))}))
    
    result= columns.reduceByKey( lambda a, b: a | b)
    answer=result.mapValues(lambda v:  round(statis.mean(v),2))
    sorted_answer=answer.sortBy(lambda x: x[1], ascending = False)
    
    for (k,v) in sorted_answer.collect():
        print("%s\t%s\n"%(k,v))
        sc.stop()
except Exception as e:
    print(e)
    sc.stop()


17031832200_4_10AM	2051.08

17031080202_0_08PM	513.62

17031081600_1_11AM	489.97

17031071200_4_11PM	482.87

17031062100_6_01AM	196.84

17031081201_0_07PM	193.12

17031242200_6_12PM	141.95

17031081700_3_03PM	141.55

17031161100_0_03PM	120.0

17031980100_4_08PM	106.04

17031081401_3_11AM	98.75

17031050600_4_01PM	97.05

17031980000_0_12PM	92.94

17031160200_5_08AM	90.0

17031081401_3_04PM	87.45

17031841900_2_02AM	82.25

17031839100_3_03PM	78.43

17031836200_3_06PM	77.25

17031830700_2_06PM	77.05

17031839200_4_04PM	75.0

17031980000_3_03AM	74.93

17031836200_5_02PM	72.9

17031031400_5_02PM	72.62

17031330100_3_01PM	72.6

17031804603_2_05PM	72.3

17031081700_4_10PM	72.17

17031812400_4_11AM	71.0

17031071200_2_06PM	70.85

17031832000_2_07PM	70.63

17031030900_3_08AM	70.45

17031160300_0_07PM	70.0

17031830700_0_05PM	69.25

17031980000_6_04AM	68.65

17031842300_0_05AM	68.25

17031030200_6_06AM	67.92

17031833000_4_09PM	67.79

17031191302_2_05PM	66.6

17031062600_5_10PM	66.5

17031411200

## 4. How many taxi trips are made under each Company and how much money has that company made
 

Output is expected to have four columns company, year, nºtrips, totalcost. 


In [1]:
%%time
import pyspark 
from operator import add as sum
import statistics as statis

sc = pyspark.SparkContext('local[*]')
try :
    
    lines = sc.textFile('Taxi_Trips_151MB.csv')
    no_empty_logs =lines.filter(lambda lines : len(lines) > 0 )
    columns=no_empty_logs.map(lambda log: log.split(';')).map(lambda v:(v[16],(v[14]),v[2]))
    filtering=columns.filter(lambda v: len(v[0])>0 and len(v[1])>0  )
    cleaning=filtering.map(lambda v:(v[0].strip(" -0123456789") + " -" + v[2][6:-11] , [1 , float(v[1].replace(',',''))]))  
    
    answer = cleaning.reduceByKey(lambda a, b: [a[0] + b[0], a[1] + b[1]] )
    sorted_answer = answer.sortBy( lambda x: x[0], ascending = False)
    
    for (k,v) in sorted_answer.collect():
        print("%s\t%d %0.2f\n"%(k,v[0],v[1]))
        sc.stop()
except Exception as e:
    print(e)
    sc.stop()

Zuha Taxi -2015 	11 149.85

Zuha Taxi -2014 	14 226.30

Zip Cab -2017 	2 13.50

Zip Cab -2016 	9 90.50

Zip Cab -2015 	7 78.95

Zip Cab -2014 	13 173.25

Zeymane Corp -2018 	6 202.10

Zeymane Corp -2017 	5 138.05

Zeymane Corp -2016 	6 123.90

Zeymane Corp -2015 	8 107.65

Zeymane Corp -2014 	10 71.85

Zeymane Corp -2013 	5 33.95

Yellow Cab -2019 	4 36.25

Yellow Cab -2018 	1519 22775.24

Yellow Cab -2017 	3700 53147.62

Yellow Cab -2016 	5673 84470.16

Yellow Cab -2015 	6404 88740.64

Yellow Cab -2014 	2910 39149.83

Valley Cab Co -2017 	13 155.50

Valley Cab Co -2016 	8 121.75

Valley Cab Co -2015 	11 130.00

Valley Cab Co -2014 	5 65.35

U Taxicab -2020 	31 1064.10

U Taxicab -2019 	6 71.30

Tyrone Henderson -2018 	10 169.66

Tyrone Henderson -2017 	7 76.25

Tyrone Henderson -2016 	5 50.10

Tyrone Henderson -2015 	14 159.65

Tyrone Henderson -2014 	5 32.35

Tyrone Henderson -2013 	4 23.40

Top Cab Affiliation -2020 	113 1994.70

Top Cab Affiliation -2019 	480 9805.35

Top Cab Affil